<a href="https://colab.research.google.com/github/gvlokesh/Classification/blob/master/Text_generation_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pwd
# Taming of the shrew.txt can be downloaded from http://www.gutenberg.org/cache/epub/2245/pg2245.txt
filename = "Taming of the shrew.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

/content


In [0]:
i=0
c=None
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))


In [0]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  134260
Total Vocab:  65


In [0]:
# prepare the dataset of input to output pairs encoded as integers
i=0
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  134160


In [0]:
import numpy as np
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)

In [0]:
from keras.utils import np_utils
y = np_utils.to_categorical(dataY)

In [0]:
print(np.shape(X),np.shape(y))

(134160, 100, 1) (134160, 64)


In [0]:
class BatchMaker():
      def __init__(self):
          
          self.i = 0  
          self.X_train=X
          self.y_train=y
                           
      def next_batch(self, batch_size):
        x = self.X_train[self.i:self.i+batch_size]
        y = self.y_train[self.i:self.i+batch_size]
        self.i = (self.i + batch_size) % (len(self.X_train))
        return x, y 

In [0]:
ch=BatchMaker()
#for i in range(7000):
       # batch = ch.next_batch(1000)
        #print(batch[1])
X1= ch.next_batch(15)       
print((X1[0]).shape)
print((X1[1]).shape)

(15, 100, 1)
(15, 64)


In [0]:
import tensorflow as tf

#LSTM parameter setup
n_steps=100
n_inputs=1
n_classes=64
n_hidden_units=256
l_r = 0.001

In [0]:

tf.reset_default_graph()

#define placeholder for input
x1 = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y1 = tf.placeholder(tf.float32, [None, n_classes])

print(x1)
print(y1)

Tensor("Placeholder:0", shape=(?, 100, 1), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 64), dtype=float32)


In [0]:
# define w and b
weights = {
    'in': tf.Variable(tf.random_normal([n_inputs,n_hidden_units])),
    'out': tf.Variable(tf.random_normal([n_hidden_units,n_classes]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1,shape=[n_hidden_units,])),
    'out': tf.Variable(tf.constant(0.1,shape=[n_classes,]))
}

In [0]:
X_in = tf.reshape(x1, [-1, n_inputs])
print ("Shape of X is" ,X_in)
X_in = tf.matmul(X_in, weights['in']) + biases['in']
print ("Shape of X_in is" ,X_in)
X_in = tf.reshape(X_in, [-1,n_steps, n_hidden_units])
print ("Shape of X_in  after reshape is" ,X_in)

Shape of X is Tensor("Reshape:0", shape=(?, 1), dtype=float32)
Shape of X_in is Tensor("add:0", shape=(?, 256), dtype=float32)
Shape of X_in  after reshape is Tensor("Reshape_1:0", shape=(?, 100, 256), dtype=float32)


In [0]:
# ###SIngle lstm
# batch_size_T  = tf.shape(X_in)[0]

# lstm_cell = tf.nn.rnn_cell.LSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)

# _init_state = lstm_cell.zero_state(batch_size_T ,  dtype=tf.float32)
# outputs,states = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=_init_state, time_major=False)

# print ("Shape of outputs  is" ,outputs)

In [0]:
#multi lstm

n_layers = 4
batch_size_T  = tf.shape(X_in)[0]

cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(num_units=n_hidden_units)
          for layer in range(n_layers)])

# _init_state = lstm_cell.zero_state(batch_size_T ,  dtype=tf.float32)-------
outputs,states = tf.nn.dynamic_rnn(cell, X_in, dtype=tf.float32)

In [0]:
outputs1 = tf.unstack(tf.transpose(outputs, [1,0,2]))

In [0]:
results = tf.matmul(outputs1[-1], weights['out']) + biases['out']

In [0]:
pred=results
print (pred)

Tensor("add_1:0", shape=(?, 64), dtype=float32)


In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y1))
train_op = tf.train.AdamOptimizer(l_r).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y1,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [0]:
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
ch=BatchMaker()
for e in range(10):
  for i in range(1400):
          batch_x, batch_y = ch.next_batch(100)  
          out=sess.run(train_op,feed_dict={x1: batch_x, y1: batch_y}) 
          if i % 100 == 0:
              print("at epoch :",e," accuracy at step :",i,"is ",sess.run(accuracy,feed_dict={x1: batch_x,y1: batch_y,}))
              
           
saver.save(sess, "./model")           
         

at epoch : 0  accuracy at step : 0 is  0.2
at epoch : 0  accuracy at step : 100 is  0.31
at epoch : 0  accuracy at step : 200 is  0.18
at epoch : 0  accuracy at step : 300 is  0.14
at epoch : 0  accuracy at step : 400 is  0.2
at epoch : 0  accuracy at step : 500 is  0.2
at epoch : 0  accuracy at step : 600 is  0.16
at epoch : 0  accuracy at step : 700 is  0.21
at epoch : 0  accuracy at step : 800 is  0.21
at epoch : 0  accuracy at step : 900 is  0.24
at epoch : 0  accuracy at step : 1000 is  0.23
at epoch : 0  accuracy at step : 1100 is  0.22
at epoch : 0  accuracy at step : 1200 is  0.28
at epoch : 0  accuracy at step : 1300 is  0.29
at epoch : 1  accuracy at step : 0 is  0.24
at epoch : 1  accuracy at step : 100 is  0.21
at epoch : 1  accuracy at step : 200 is  0.28
at epoch : 1  accuracy at step : 300 is  0.24
at epoch : 1  accuracy at step : 400 is  0.25
at epoch : 1  accuracy at step : 500 is  0.29
at epoch : 1  accuracy at step : 600 is  0.21
at epoch : 1  accuracy at step : 700 

'./model'



---



---


**Inference starts here**


---



---



In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))


In [0]:
import sys
start = np.random.randint(0, len(dataX)-1)
print(start)
pattern = dataX[start]
len(pattern)


58516


100

In [0]:
with tf.Session() as sess:
  
  saver.restore(sess, "./model")
      
    ## Now create a for loop that 
  for i in range(1000):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)   
        y_pred = sess.run(pred, feed_dict={x1: x})
        index = np.argmax(y_pred)
        result = int_to_char[index]
#         print(result)
        sys.stdout.write(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
  print ("\nDone."    )     
	      
        

INFO:tensorflow:Restoring parameters from ./model
fe, and the manke of mine

   pet. why that is not sister then the eldest:
i will go with his to my selfe, and the manke of my selfe,
and the may be seuenge and behauiour,
and the is my selfe, and the manke of mine

   pet. why that is not sister then the eldest:
i will go with his to my selfe, and the manke of my selfe,
and the may be seuenge and behauiour,
and the is my selfe, and the manke of mine

   pet. why that is not sister then the eldest:
i will go with his to my selfe, and the manke of my selfe,
and the may be seuenge and behauiour,
and the is my selfe, and the manke of mine

   pet. why that is not sister then the eldest:
i will go with his to my selfe, and the manke of my selfe,
and the may be seuenge and behauiour,
and the is my selfe, and the manke of mine

   pet. why that is not sister then the eldest:
i will go with his to my selfe, and the manke of my selfe,
and the may be seuenge and behauiour,
and the is my selfe, a